In [1]:
import keras
import numpy as np
print (keras.__version__)

Using TensorFlow backend.


2.2.0


In [2]:
import tensorflow as tf
print (tf.__version__)

1.9.0


In [3]:


from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.engine.topology import Layer
from keras.optimizers import RMSprop
import numpy as np

class MyLayer(Layer):

    def __init__(self,output_dim,thre, **kwargs):
        self.output_dim = output_dim
        self.threshold=thre
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        #self.kernel = self.add_weight(name='kernel', shape=(input_shape[1], self.output_dim),initializer='uniform',trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def getPaddedFilter(self,myfilter,padNumOfOnes):
        numRow = K.shape(myfilter)[0]
        
        pad = K.ones((numRow,padNumOfOnes))
        
        myfilter2=K.concatenate( [pad, myfilter],axis=1)
        
        return myfilter2
        
    def call(self, x):
        myfilter = x[1]
        value = x[0]
        
        inputDim=K.shape(value)[-1]
        filterDim = K.shape(myfilter)[-1]
        
        #if inputDim > filterDim, pad 1 on the left of filter
        padDim = K.switch(K.less(filterDim,inputDim),inputDim-filterDim,0 )
        myfilter=self.getPaddedFilter(myfilter,padDim)
        #if inputDim < filterDim, shift filter left 
        shiftLeft = K.switch(K.greater(filterDim,inputDim) ,filterDim-inputDim,0)
        myfilter2 = myfilter[:,shiftLeft:]
        
        myfilter2=K.cast(myfilter2,"float32")
        myfilter2=K.abs(myfilter2)
        myfilter2=myfilter2>self.threshold
        myfilter2=K.cast(myfilter2,"float32")
        
        #return padDim
        return (value*myfilter2)
        #return myfilter2

    def get_config(self):
        config = {'output_dim': self.output_dim, 
                  'thre': self.threshold}
        base_config = super(MyLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
    
    
longestTenor=40    
inputsize=7
filtersize=inputsize
outputsize=4
inputs = Input(shape=(inputsize,))
#mask=Input(shape=(filtersize,))
x = Dense(10, activation="relu",kernel_initializer="one")(inputs)
predict = Dense(inputsize, activation="relu",kernel_initializer="one")(x)
predict = Dense(outputsize, activation="relu",kernel_initializer="one")(x)
predict=MyLayer(outputsize,np.exp(-1*longestTenor).astype("float32"))([predict,inputs])

model = Model(inputs=inputs, outputs=predict)

In [4]:
model.compile(optimizer=RMSprop(lr=0.01),loss="mse",metrics=["mae"])

In [5]:
ii = np.array([[200,3,5,6,0,1,0]
               ,[-1,2,0.0001,0,5,6,8]
              ])
output = np.ones(shape=(2,outputsize))

In [6]:
callbackLst = [
    keras.callbacks.TensorBoard(
        log_dir='./model_log_dir/keraFilterLayer', #Log files will be written at this location.
        histogram_freq=1, #Records activation histograms every 1 epoch R
        
    ),
    keras.callbacks.ModelCheckpoint(
            filepath='./modelTrain/modelkeraFilterLayer2',
            monitor='val_loss',
            save_best_only=True,
    )
]

In [7]:
history = model.fit(x=ii,y=output,epochs=5,callbacks=callbackLst,validation_split=0.2)

Train on 1 samples, validate on 1 samples
Epoch 1/5
1/1 [==============================] - 0s 185ms/step - loss: 2309101.0000 - mean_absolute_error: 1075.0000 - val_loss: 28344.2383 - val_mean_absolute_error: 146.0343
Epoch 2/5
1/1 [==============================] - 0s 3ms/step - loss: 2029778.7500 - mean_absolute_error: 1007.9171 - val_loss: 27440.9688 - val_mean_absolute_error: 143.6609
Epoch 3/5
1/1 [==============================] - 0s 3ms/step - loss: 1852041.6250 - mean_absolute_error: 962.7996 - val_loss: 26726.5938 - val_mean_absolute_error: 141.7439
Epoch 4/5
1/1 [==============================] - 0s 3ms/step - loss: 1716258.1250 - mean_absolute_error: 926.8524 - val_loss: 26120.7324 - val_mean_absolute_error: 140.0890
Epoch 5/5
1/1 [==============================] - 0s 2ms/step - loss: 1604492.1250 - mean_absolute_error: 896.1818 - val_loss: 25586.8750 - val_mean_absolute_error: 138.6076


In [19]:

k=model.predict(ii.reshape(-1,inputsize))

In [20]:
k

array([[1738.0345  ,    0.      , 1738.0345  ,    0.      ],
       [   0.      ,  111.949684,  100.581535,    0.      ]],
      dtype=float32)

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           80          input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 4)            44          dense_1[0][0]                    
__________________________________________________________________________________________________
my_layer_1 (MyLayer)            ((None, 4), 4)       0           dense_3[0][0]                    
                                                                 input_1[0][0]                    
Total para

In [11]:
from keras.models import load_model

In [12]:
model2 = load_model('./modelTrain/modelkeraFilterLayer2',custom_objects={'MyLayer': MyLayer})

In [18]:
ii = np.array([[200,3,5,6,0,1,0.00]
               ,[-1,2,0.0001,0,5,6,0]
              ])

model2.predict(ii.reshape(-1,inputsize))

array([[1738.0345  ,    0.      , 1738.0345  ,    0.      ],
       [   0.      ,  111.949684,  100.581535,    0.      ]],
      dtype=float32)

In [ ]:
p=np.ones((2,0))
v = np.random.randint(2,5,size=(2,3))
np.concatenate([p,v],axis=1)[:,:10]

In [ ]:


ii = np.array([[2,3,5,6,0,0,0],[1,2,3,4,5,6,7]])
#mask_ = ii[:,inputsize-filtersize:]
mask_ = np.concatenate( [ii, np.zeros((2,2))],axis=1)
mask_

In [ ]:
from keras.layers import Activation, Dense
from keras.layers import Multiply
class MaskedDenseLayer(Layer):
    def __init__(self, output_dim, activation, **kwargs):
        self.output_dim = output_dim
        super(MaskedDenseLayer, self).__init__(**kwargs)
        self._activation = Activation(activation)
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[0][1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(MaskedDenseLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, l):
        self.x = l[0]
        self._mask = l[1][1]
        print('self._mask', self._mask)
        kernel_shape = K.shape(self.kernel).eval(session=K.get_session())
        
        masked = Multiply()([self.kernel, self._mask])
        self._output = K.dot(self.x, masked)
        return self._activation(self._output)

    
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.output_dim)

In [ ]:
import numpy as np
x = np.arange(1,10)

In [ ]:
x[::-1]